# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('Orders.csv')
print(orders.shape)
orders.head()


(397924, 14)


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# Sub Problem 1: How to aggregate the amount_spent for unique customers?
spending = orders.groupby(['CustomerID'])['amount_spent'].sum().reset_index()
spending.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [4]:
# Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
# Use pd quantile or pd qcut, or pd cut? https://kanoki.org/2020/04/20/how-to-create-bins-in-pandas-using-cut-and-qcut/
spending.hist(bins=4)
spending.describe()

,CustomerID,amount_spent
count,4339.000000,4339.000000
mean,15299.936852,2053.793018
std,1721.889758,8988.248381
min,12346.000000,0.000000
25%,13812.500000,307.245000
50%,15299.000000,674.450000
75%,16778.500000,1661.640000
max,18287.000000,280206.020000


In [5]:
spending['Type'] = pd.qcut(spending['amount_spent'], [0, 0.75, 0.95, 1], labels = ["Normal","Preferred Customers","VIP"])
spending.head()

,CustomerID,amount_spent,Type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred Customers
2,12348,1797.24,Preferred Customers
3,12349,1757.55,Preferred Customers
4,12350,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [17]:
spending['Country'] = orders['Country']
spending.Country.unique()
# spending.sort_values(by = 'amount_spent', ascending= False).head()
spending.groupby(['Type','Country'])['Type'].count()

# The largest amount of vip clients is in the UK

Type                 Country       
Normal               Australia            9
                     EIRE                15
                     France              19
                     Germany             29
                     Netherlands          2
                     Norway              56
                     Spain                3
                     Switzerland          5
                     United Kingdom    3116
Preferred Customers  Australia            4
                     EIRE                 8
                     France               8
                     Germany              7
                     Norway               8
                     Spain                1
                     Switzerland          1
                     United Kingdom     831
VIP                  Australia            1
                     EIRE                 2
                     Norway               9
                     Spain                1
                     United Kingdom     

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [7]:
# The largest amount of vip clients  + Preferred is in the UK